### загрузка данных

In [28]:
# pip install xlrd     # for reading excel files
# pip install openpyxl # for reading excel files

In [3]:
import pandas as pd

In [4]:
subtitles = pd.read_csv('subtitles.txt')
english_level_subtitles = pd.read_csv('english_level_subtitles.txt')
subtitles_all_A2 = pd.read_csv('subtitles_all_A2.txt')
subtitles_all_B1 = pd.read_csv('subtitles_all_B1.txt')
subtitles_all_B2 = pd.read_csv('subtitles_all_B2.txt')
subtitles_all_C1 = pd.read_csv('subtitles_all_C1.txt')
movies_labels = pd.read_excel('English_level/English_scores/movies_labels.xlsx').drop('id',axis=1)

In [5]:
subtitles['subtitles']=1
english_level_subtitles['el_subtitles']=1
subtitles_all_A2['subtitles_all']='A2'
subtitles_all_B1['subtitles_all']='B1'
subtitles_all_B2['subtitles_all']='B2'
subtitles_all_C1['subtitles_all']='C1'
subtitles_all = pd.concat((subtitles_all_A2,subtitles_all_B1,subtitles_all_B2,subtitles_all_C1))

In [6]:
# удаляем расширение .str в трёх датасетах
for df in (subtitles,english_level_subtitles,subtitles_all):
    df['Movie']=df['Movie'].apply(lambda x: x[:-4])

In [7]:
# число полных дубликатов во всех четырёх датасетах
for df in (subtitles, 
           english_level_subtitles, 
           movies_labels, 
           subtitles_all):
    print(df.duplicated().sum())

0
0
2
0


In [8]:
# задублированные фильмы 'movies_labels' 
movies_labels[movies_labels.duplicated(keep=False)]

,Movie,Level
38,Powder(1995),B1
43,Inside_out(2015),B1
44,Inside_out(2015),B1
68,Powder(1995),B1


In [9]:
# удаление полных дубликатов 
movies_labels = movies_labels.drop_duplicates()

In [10]:
# число дубликатов по признаку Movie в четырёх датасетах
for df in (subtitles, 
           english_level_subtitles, 
           movies_labels, 
           subtitles_all):
    print(df['Movie'].duplicated().sum())

0
0
2
0


In [11]:
# задублированные фильмы 'movies_labels' 
movies_labels[movies_labels['Movie'].duplicated(keep=False)]

,Movie,Level
75,The_blind_side(2009),B2
83,The_terminal(2004),B1
84,The_blind_side(2009),B1
99,The_terminal(2004),"A2/A2+, B1"


Объединим значения уровня у одинаковых фильмов. Лишнюю строку удалим.

In [12]:
movies_labels.loc[75,'Level'] = 'B1,B2'
movies_labels.drop([83,84], inplace=True)

In [13]:
# названия фильмов делаем индексами во всех датасетах
for df in (subtitles, 
           english_level_subtitles, 
           subtitles_all,
           movies_labels):
    df.set_index('Movie', inplace=True)

In [14]:
movies_labels.columns=['movies_labels']

In [15]:
# сводная со всеми упомянутыми в проекте фильмами
subtitles_pivot = subtitles.join(english_level_subtitles, how='outer')\
                           .join(subtitles_all,           how='outer')\
                           .join(movies_labels,           how='outer')

In [16]:
subtitles_pivot.head(3)

,subtitles,el_subtitles,subtitles_all,movies_labels
Movie,,,,
10_Cloverfield_lane(2016),1.0,1.0,NaN,B1
10_things_I_hate_about_you(1999),1.0,1.0,NaN,B1
A_knights_tale(2001),1.0,1.0,NaN,B2


In [17]:
subtitles_pivot.shape[0]

287

In [18]:
subtitles_pivot.count() # проверка кол-ва фильмов в каждом датасете

subtitles         86
el_subtitles     115
subtitles_all    163
movies_labels    237
dtype: int64

Всё соответсвует (с учётом четырёх убранных дубликатов).

### анализ сводной таблицы

In [19]:
# кол-во различных позиций в 'subtitles' и 'el_subtitles'
subtitles_pivot.query('(subtitles.isna() & ~el_subtitles.isna()) | (~subtitles.isna() & el_subtitles.isna())').shape[0]

31

In [20]:
# различия в 'subtitles' и 'el_subtitles'
subtitles_pivot.query('(subtitles.isna() & ~el_subtitles.isna()) | (~subtitles.isna() & el_subtitles.isna())')

,subtitles,el_subtitles,subtitles_all,movies_labels
Movie,,,,
Banking_On_Bitcoin(2016),NaN,1.0,NaN,C1
Breaking_Bad_The_Movie(2017),NaN,1.0,NaN,NaN
Bren▒.Brown.The.Call.to.Courage.2019.720.NF.720p.DDP.5.1.x264-CafeFlix,NaN,1.0,NaN,NaN
Bridget_Joness_Baby,NaN,1.0,NaN,C1
Cars(2006),NaN,1.0,NaN,B1
Casper,NaN,1.0,NaN,NaN
Charlie_and_the_Chocolate_Factory,NaN,1.0,NaN,B1
Cinderella(1950),NaN,1.0,NaN,B1
Despicable_Me(2010),NaN,1.0,NaN,B2


Задублировано название фильма "Its_a_wonderful_life(1946).srt". Исправим в "subtitles":

In [21]:
subtitles.rename(index={'It_s_a_wonderful_life(1946)':'Its_a_wonderful_life(1946)'}, inplace=True)

Переделаем сводную таблицу чтобы ещё раз проверить.

In [22]:
# сводная со всеми упомянутыми в проекте фильмами (после исправления наименования одного фильма в 'subtitles')
subtitles_pivot = subtitles.join(english_level_subtitles, how='outer')\
                           .join(subtitles_all,           how='outer')\
                           .join(movies_labels,           how='outer')

In [23]:
# кол-во различных позиций в 'subtitles' и 'el_subtitles' (после исправления)
subtitles_pivot.query('(subtitles.isna() & ~el_subtitles.isna()) | (~subtitles.isna() & el_subtitles.isna())').shape[0]

29

In [24]:
# число фильмов, которые присутствуют в базах subtitles_all и subtitles_all; их значения уровня английского идентичны
subtitles_pivot.query('subtitles_all==movies_labels' ).shape[0]

123

In [25]:
# число фильмов, которые присутствуют в базах subtitles_all и movies_labels; их значения уровня английского отличаются
subtitles_pivot.query('subtitles_all!=movies_labels & ~subtitles_all.isna() & ~movies_labels.isna()').shape[0]

0

In [26]:
# число фильмов, которые присутствуют только в одной из баз subtitles_all или movies_labels
subtitles_pivot.query('subtitles_all!=movies_labels').shape[0]

163

Вывод:
1. в 'el_subtitles' все фильмы из 'subtitles' (86 шт.), а также дополнительно ещё 29 уникальных. Всего в 'el_subtitles' - 115 шт. В работе необходимо использовать 'el_subtitles', как более обширную базу и включающую в себя маленькую базу;
2. ...